<span STYLE="font-size:150%"> 
    Segment microCT scans
</span>

Docker image: gnasello/slicer-env:2023-07-06 \
Latest update: 10 March 2023

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

# Load libraries

In [ ]:
import pyslicer as ps
import slicer
from pathlib import Path
import pandas as pd
import numpy as np
import os
from ScreenCapture import ScreenCaptureLogic

# Volume input

## Load `.nrrd` file into 3D Slicer

Write the path of the `.nrrd` file and load it to Slicer

In [ ]:
# this cell is tagged 'parameters'
volume_file = 'microCT_volume/microCT_volume.nrrd'
output_dir_path = 'segmented_volumes'

In [ ]:
path = Path(volume_file)

# Remove image numbering _0000, _0001 ...
filename_output = path.stem[:-4]

In [ ]:
masterVolumeNode = slicer.util.loadNodeFromFile(volume_file)

Print spacing

In [ ]:
## mm
masterVolumeNode.GetSpacing()

Make ```segmented_volumes``` folder

In [ ]:
output_directory = Path(output_dir_path)

output_directory.mkdir(parents=True, exist_ok=True)

## Adjust window/level (brightness/contrast)

Adjust the image window/level from the GUI. It is enough to select a region where the signal is present ([youtube](https://slicer.readthedocs.io/en/latest/user_guide/user_interface.html#adjusting-image-window-level)). 

After, get the Window and Level properties from the displayNode associate to the VolumeNode and apply them programmatically anytime you run again the same image.

For more information on the relationship between window/level and brightness/contrast, see [Window and Level Contrast Enhancement](http://fisica.ciens.ucv.ve/curs/dipcourse/html/one-oper/window-level/front-page.html)

In [ ]:
displayNode = masterVolumeNode.GetDisplayNode()
print('displayNode.SetWindow(' + str(displayNode.GetWindow()) + ')')
print('displayNode.SetLevel(' + str(displayNode.GetLevel()) + ')')

In [ ]:
# displayNode = masterVolumeNode.GetDisplayNode()
# displayNode.AutoWindowLevelOff()
# displayNode.SetWindow(25885.0)
# displayNode.SetLevel(18464.5)

# Take screenshots of original image

In [ ]:
screenshots_directory = output_directory / 'Screenshots'

screenshots_directory.mkdir(parents=True, exist_ok=True)

In [ ]:
# Function to capture a screenshot of a specific view
def capture_view(view, output_path):
    screen_capture_logic = ScreenCaptureLogic()
    screen_capture_logic.captureImageFromView(view, output_path)

# Get the layout manager
layout_manager = slicer.app.layoutManager()

# Capture screenshots of the Red, Yellow, Green, and 3D views
views = {
    'Red': layout_manager.sliceWidget('Red').sliceView(),
    'Yellow': layout_manager.sliceWidget('Yellow').sliceView(),
    'Green': layout_manager.sliceWidget('Green').sliceView(),
    # '3D': layout_manager.threeDWidget(0).threeDView()
}

for view_name, view_widget in views.items():
    output_path = os.path.join(str(screenshots_directory), f'{view_name}_original_view.png')
    capture_view(view_widget, output_path)
    print(f'Screenshot saved: {output_path}')

print('Screenshots captured successfully.')


# Create segmentationNode

## Create segmentation-related nodes

Create segmentation node

In [ ]:
segmentationNode = ps.segmentation.segmentationNode(name='Segmentation')

Create temporary segment editor to get access to effects

In [ ]:
segmentEditorWidget, segmentEditorNode = ps.segmentation.segmentEditorWidget(segmentationNode = segmentationNode, 
                                                                             masterVolumeNode = masterVolumeNode)

# Thresholding

Load thresholding values

In [ ]:
df_greyvalues = pd.read_csv(str(output_directory / 'segments_greyvalues.csv'))

segments_greyvalues = df_greyvalues.to_dict()
segments_greyvalues

## Create segments by thresholding

In [ ]:
ps.segmentation.segments_by_thresholding(dict((k, segments_greyvalues[k]) for k in ('Bone', 'Scaffold')) , 
                                         segmentationNode,
                                         segmentEditorNode,
                                         segmentEditorWidget)

Set segments color

In [ ]:
segments_color = {
    "Bone": (0.9450980392156862, 0.8392156862745098, 0.5686274509803921), # "Bone" color in Slicer
    "Scaffold": (1, 1, 225/250), # "Bone" color in Slicer
    }

segments_color

In [ ]:
ps.segmentation.set_segments_color(segments_color, segmentationNode)

# Operation on segments

## Manual fix of the segmentation

Sometimes it might be necessary to remove speckles at the image boundaries. If so, use the `scissor` tool in the `Segment Editor` before proceeding with the rest of the script. 

## Keep largest islands

In [ ]:
# segment_name = 'Scaffold'
# minimum_size = 2000 #number of voxels

In [ ]:
# ps.segmentation.keep_largest_island(minimum_size, 
#                                     segment_name, 
#                                     segmentEditorNode,
#                                     segmentEditorWidget)

## Remove small islands

REMOVE_SMALL_ISLANDS operation from the [SegmentEditorIslandsEffect](https://github.com/Slicer/Slicer/blob/294ef47edbac2ccb194d5ee982a493696795cdc0/Modules/Loadable/Segmentations/EditorEffects/Python/SegmentEditorIslandsEffect.py#L402)

In [ ]:
# segment_name = 'Bone'
# minimum_size = 20 #number of voxels

In [ ]:
# ps.segmentation.remove_small_islands(minimum_size, 
#                                      segment_name, 
#                                      segmentEditorNode,
#                                      segmentEditorWidget)

## Smoothing thresholded segment

In [ ]:
# segment_name = 'Bone'
# gaussiaSD_mm = 0.001 #units are in millimiters

In [ ]:
# ps.segmentation.gaussian_smoothing(gaussiaSD_mm, 
#                                    segment_name, 
#                                    segmentEditorNode, 
#                                    segmentEditorWidget)

In [ ]:
# segment_name = 'Scaffold'
# gaussiaSD_mm = 0.001 #units are in millimiters

In [ ]:
# ps.segmentation.gaussian_smoothing(gaussiaSD_mm, 
#                                    segment_name, 
#                                    segmentEditorNode, 
#                                    segmentEditorWidget)

# Save screenshots

In [ ]:
# Function to capture a screenshot of a specific view
def capture_view(view, output_path):
    screen_capture_logic = ScreenCaptureLogic()
    screen_capture_logic.captureImageFromView(view, output_path)

# Get the layout manager
layout_manager = slicer.app.layoutManager()

# Capture screenshots of the Red, Yellow, Green, and 3D views
views = {
    'Red': layout_manager.sliceWidget('Red').sliceView(),
    'Yellow': layout_manager.sliceWidget('Yellow').sliceView(),
    'Green': layout_manager.sliceWidget('Green').sliceView(),
    # '3D': layout_manager.threeDWidget(0).threeDView()
}

for view_name, view_widget in views.items():
    output_path = os.path.join(str(screenshots_directory), f'{view_name}_segmented_view.png')
    capture_view(view_widget, output_path)
    print(f'Screenshot saved: {output_path}')

print('Screenshots captured successfully.')


# Export segments

## As seg.nrrd file (labelmap node)

Export a selection of segments (identified by their names), from [slicer tutorial](https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html#export-labelmap-node-from-segmentation-node)

Bone LabelmapNode

In [ ]:
seg_name = 'Bone'

In [ ]:
mineralized_labelmap = ps.segmentation.individual_segment_to_labelmapNode(segmentName = seg_name,
                                                                          segmentationNode = segmentationNode,
                                                                          volumeNode = masterVolumeNode)

In [ ]:
filename_output = seg_name + '.seg.nrrd'

slicer.util.exportNode(mineralized_labelmap, output_directory / filename_output)

In [ ]:
seg_name = 'Scaffold'

In [ ]:
mineralized_labelmap = ps.segmentation.individual_segment_to_labelmapNode(segmentName = seg_name,
                                                                          segmentationNode = segmentationNode,
                                                                          volumeNode = masterVolumeNode)

In [ ]:
filename_output = seg_name + '.seg.nrrd'

slicer.util.exportNode(mineralized_labelmap, output_directory / filename_output)